In [1]:
import yaml

In [23]:
with open('dags/configs/conf.yaml', "r") as config_file:
    config = yaml.safe_load(config_file)

In [24]:
config

{'connection': {'host': '172.28.0.1',
  'db': 'dv2',
  'port': 5432,
  'user': 'postgres',
  'pw': 'postgres',
  'schemas': {'stage': 'stage', 'edwh': 'vault'}},
 'hubs': {'hub_tax_bundle': {'hk': 'hk_tax_bundle',
   'overwrite': False,
   'src': {'table': 'v_report_taxation_bundle',
    'bk': 'tax_bundle_id',
    'type': 'int'}},
  'hub_ziffer': {'hk': 'hk_ziffer',
   'overwrite': False,
   'src': {'table': 'v_report_dict_ziffer', 'bk': 'ziffer', 'type': 'int'}}}}

# hubs


## creator

In [5]:
sql= open('./dags/sql/hub_creator.sql','r').read()

In [20]:
sql

'CREATE TABLE {vault}.{hub} (\n\t{hk} varchar(256) NOT NULL,\n\tload_dts timestamp NULL,\n\trec_src varchar(256) NULL,\n\t{bk} {bk_type} NULL,\n\tCONSTRAINT {hub}_pkey PRIMARY KEY ({hk})\n);'

In [30]:
conf=config

In [13]:
hub='hub_tax_bundle'
params = {'schema':conf['connection']['schemas']['edwh'],
          'hub':hub,
          'hk':conf['hubs'][hub]['bk'],
          'bk':conf['hubs'][hub]['src']['bk'],
          'bk_type':conf['hubs'][hub]['src']['type']
          }

In [19]:
sql.format(**params)

KeyError: 'vault'

## loader

In [42]:
sql= open('./dags/sql/hub_loader.sql','r').read()

In [43]:
params = {'bk_src':conf['hubs'][hub]['src']['bk'],
          'rec_src':conf['hubs'][hub]['src']['table'],
          'schema_src':conf['connection']['schemas']['stage'],
          'table_src':conf['hubs'][hub]['src']['table'],
          'schema_tgt':conf['connection']['schemas']['edwh'],
          'table_tgt':hub,
          'hk_tgt':conf['hubs'][hub]['hk'],
          'bk_tgt':conf['hubs'][hub]['src']['bk']
         }

In [44]:
sql.format(**params)

'with cte (hk,load_dts,rec_src,bk) as (\n\tselect md5(trim(src.tax_bundle_id::varchar(256))) \n\t,  current_timestamp \n\t,  v_report_taxation_bundle\n\t, src.tax_bundle_id\n\tfrom stage.v_report_taxation_bundle src\n)\ninsert into vault.hub_tax_bundle (hk_tax_bundle,load_dts,rec_src,tax_bundle_id)\nselect distinct \n\thk\n  , load_dts\n  , rec_src\n  , bk\nfrom cte\nwhere not exists \n(\n\tselect 1 from vault.hub_tax_bundle tgt\n\twhere cte.hk = tgt.hk_tax_bundle\n);'

# links

In [45]:
with open('dags/configs/conf.yaml', "r") as config_file:
    conf = yaml.safe_load(config_file)

In [46]:
conf

{'connection': {'host': '172.28.0.1',
  'db': 'dv2',
  'port': 5432,
  'user': 'postgres',
  'pw': 'postgres',
  'schemas': {'stage': 'stage', 'edwh': 'vault'}},
 'hubs': {'hub_tax_bundle': {'hk': 'hk_tax_bundle',
   'overwrite': False,
   'src': {'table': 'v_report_taxation_bundle',
    'bk': 'tax_bundle_id',
    'type': 'int'}},
  'hub_ziffer': {'hk': 'hk_ziffer',
   'overwrite': False,
   'src': {'table': 'v_report_dict_ziffer', 'bk': 'ziffer', 'type': 'int'}}},
 'links': {'l_taxbundle_ziffer': {'hk': 'hk_l_taxbundle_ziffer',
   'overwrite': False,
   'src': {'table': 'v_report_ziffer',
    'bks': {'tax_bundle_id': {'type': 'int'}, 'ziffer': {'type': 'int'}}}}}}

## link creator

In [72]:
sql= open('./dags/sql/link_creator.sql','r').read()

In [73]:
sql

'CREATE TABLE {schema}.{link} (\n\t{hk} varchar(256) NOT NULL,\n    load_dts timestamp NULL,     \n    {bk_list}\n    CONSTRAINT {link}_pkey PRIMARY KEY ({hk})\n);'

In [50]:
link='l_taxbundle_ziffer'

In [52]:
link_bks_dict=conf['links'][link]['src']['bks']

In [53]:
link_bks_dict

{'tax_bundle_id': {'type': 'int'}, 'ziffer': {'type': 'int'}}

In [83]:
link_bks=''
for key,val in link_bks_dict.items():
    #print(key,val)
    link_bks += key + ' ' + val['type'] +' NOT NULL ,'    
link_bks

'tax_bundle_id int NOT NULL ,ziffer int NOT NULL ,'

In [84]:
params = {
    'schema':conf['connection']['schemas']['edwh'],
    'link':link,
    'hk':conf['links'][link]['hk'],
    'bk_list':link_bks
}

In [85]:
sql.format(**params)

'CREATE TABLE vault.l_taxbundle_ziffer (\n\thk_l_taxbundle_ziffer varchar(256) NOT NULL,\n    load_dts timestamp NULL,     \n    tax_bundle_id int NOT NULL ,ziffer int NOT NULL ,\n    CONSTRAINT l_taxbundle_ziffer_pkey PRIMARY KEY (hk_l_taxbundle_ziffer)\n);'

## link loader

In [131]:
sql= open('./dags/sql/link_loader.sql','r').read()

In [90]:
link='l_taxbundle_ziffer'

In [91]:
link_bks_dict=conf['links'][link]['src']['bks']

In [133]:
bk_concat_str="CONCAT_WS('|'"
bk_list=''
bk_list2=''
for key,val in link_bks_dict.items():    
    bk_list += 'src.'+key+','    
    bk_list2 += key+','    
    bk_concat_str+=f",trim(src.{key}::varchar(256))"

bk_list2=bk_list2[:-1] 
bk_list=bk_list[:-1]

In [134]:
bk_list

'src.tax_bundle_id,src.ziffer'

In [121]:
bk_list2

'tax_bundle_id,ziffer'

In [126]:
params = {
    'bk_concat_str':bk_concat_str,
    'bk_list':bk_list,
    'bk_list2':bk_list2,
    'schema_src':conf['connection']['schemas']['stage'],
    'table_src':conf['links'][link]['src']['table'],
    'schema_tgt':conf['connection']['schemas']['edwh'],
    'table_tgt':link,
    'hk_tgt': conf['links'][link]['hk']
}

In [132]:
print(sql.format(**params))

with cte (hk,load_dts,rec_src,bk) as (
	select md5(CONCAT_WS('|',trim(src.tax_bundle_id::varchar(256)),trim(src.ziffer::varchar(256))) 
	,  current_timestamp 
	, src.tax_bundle_id,src.ziffer,
	from stage.v_report_ziffer src
)
insert into vault.l_taxbundle_ziffer (hk_l_taxbundle_ziffer,load_dts,tax_bundle_id,ziffer)
select distinct hk
  , load_dts
  , tax_bundle_id,ziffer
from cte
where not exists 
(
	select 1 from vault.l_taxbundle_ziffer tgt
	where cte.hk = tgt.hk_l_taxbundle_ziffer
);
